## Read graph of metafeatures

In [5]:
import networkx as nx
g_x = nx.read_gpickle("encoded_fasttext.gpickle")
#g = nx.read_gpickle("encoded_features.gpickle")
#g = nx.read_gpickle("siimple.gpickle")
order = 0
for x,n in sorted(g_x.nodes(data=True)):
    t = n['tipo']
    if t == "dataset":
        n['tipo'] = 0
    if t == "feature dataset":
        n['tipo'] = 1
    if t == "literal dataset":
        n['tipo'] = 2
    if t == "attribute":
        n['tipo'] = 3
    if t == "feature attribute":
        n['tipo'] = 4
    if t == "literal attribute":
        n['tipo'] = 5  
    n['order']=order
    order+=1
    
datasets = [x for x,y in g_x.nodes(data=True) if y['tipo']==0]
order = [y['order'] for x,y in g_x.nodes(data=True) if y['tipo']==0]
map_order = dict(zip(datasets,order))
map_order['DS_1']

374

## Get 60% train, 20% val and 20% test from csv

In [6]:
import numpy as np
import pandas as pd

def read_dataset(path,drop_columns=None,keep_columns=None):
    #get rid of useless columns
    csv_data = pd.read_csv(path)
    
    if keep_columns != None:
        #keep only these columns
        return csv_data.filter(items=keep_columns)
    
    if drop_columns!= None:
        #drop these and keep the rest
        return csv_data.drop(drop_columns, axis=1)
    
    #finally, didn't drop or filter any column
    return csv_data     


In [7]:
from random import randrange
def get_samples(df):
    train_mask = []
    val_mask = []
    test_mask = []
    train_nodes = []
    test_nodes = []
    for index, row in df.iterrows():
        if row[2] == 1:
            if randrange(0, 10) <= 7:
                train_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
            else:
                test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 if index % 2 == 0:
#                     val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 else:
#                     test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
        else:
            if randrange(0, 10) > 8:
                if randrange(0, 30) <= 24:
                    train_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
                else:
                    test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     if index % 2 == 0:
#                         val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     else:
#                         test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
    return train_mask,test_mask
#     return train_mask,val_mask,test_mask

In [38]:
df_matching = read_dataset("./openml_203ds_datasets_matching.csv",keep_columns=["'dataset1_id'", "'dataset2_id'","'matching_topic'"]);
# train_mask,val_mask,test_mask = get_samples(df_matching)
# print("Train samples: "+str(len(train_mask)) + " Val samples: "+str(len(val_mask))+ " Test samples: "+str(len(test_mask)))
train_mask,test_mask = get_samples(df_matching)
print("Train samples: "+str(len(train_mask)) + " Test samples: "+str(len(test_mask)))


Train samples: 2116 Test samples: 432


In [33]:
test_mask

[[374, 417, -1],
 [374, 503, -1],
 [374, 377, -1],
 [571, 539, -1],
 [571, 419, -1],
 [576, 534, -1],
 [576, 429, -1],
 [576, 512, 1],
 [417, 464, 1],
 [417, 496, 1],
 [417, 525, -1],
 [417, 406, -1],
 [417, 378, -1],
 [418, 486, 1],
 [418, 492, -1],
 [418, 556, 1],
 [418, 575, -1],
 [418, 442, -1],
 [418, 443, -1],
 [464, 560, -1],
 [464, 451, -1],
 [466, 470, -1],
 [466, 548, -1],
 [466, 401, -1],
 [470, 556, 1],
 [470, 384, 1],
 [470, 434, -1],
 [470, 441, 1],
 [476, 510, 1],
 [476, 410, -1],
 [486, 496, 1],
 [486, 448, -1],
 [491, 551, 1],
 [491, 432, -1],
 [491, 453, -1],
 [495, 520, -1],
 [495, 538, 1],
 [495, 563, -1],
 [495, 387, -1],
 [495, 426, -1],
 [495, 453, -1],
 [495, 462, -1],
 [495, 512, 1],
 [496, 499, -1],
 [496, 508, 1],
 [496, 441, 1],
 [499, 523, -1],
 [501, 402, -1],
 [508, 472, -1],
 [508, 564, -1],
 [508, 403, -1],
 [515, 520, -1],
 [515, 440, -1],
 [515, 446, -1],
 [520, 493, -1],
 [520, 498, -1],
 [520, 560, -1],
 [520, 415, -1],
 [521, 538, -1],
 [521, 533, 

## Deep graph library

In [9]:
import dgl
#convert from networkx to graph deep library format
g = dgl.DGLGraph()
#gdl.from_networkx(g,['vector'])
g.from_networkx(g_x,node_attrs=['tipo','vector'], edge_attrs=None)

Using backend: pytorch


In [10]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='vector', out='m')
gcn_reduce = fn.sum(msg='m', out='vector')

## Helpers

In [11]:
def resultSet_train(features,mask):
    v1 = []
    v2 = []
    labels = []
    for n in mask:
        v1.append(features[n[0]])
        v2.append(features[n[1]])
        if n[2] == 0:
            n[2] = -1
        labels.append(n[2])
    return th.stack(v1),th.stack(v2),th.tensor(labels)

def resultSet_train_softmax(features,mask,labels):
    v1 = []
    labels_out = [] 
    loaded = []
    for n in mask:
        if n[0] not in loaded:
            loaded.append(n[0])
            v1.append(features[n[0]])
            labels_out.append(labels[n[0]])
        if n[1] not in loaded:
            loaded.append(n[1])
            v1.append(features[n[1]])
            labels_out.append(labels[n[0]])
    return th.stack(v1),th.tensor(labels_out)

from scipy.spatial.distance import cosine
def resultSet_eval(features,mask):
    indices = []
    labels = []
    for n in mask:
        resultCos = (1 - cosine(features[n[0]],features[n[1]]))
        # index 0 is not similar, index 1 is similar
        result = th.tensor([1 - resultCos , resultCos])
        # get the index that is greater
        out = th.argmax(result)
        indices.append(out)
        labels.append(n[2])
    return th.tensor(indices),th.tensor(labels)

def evaluate(model, g, features, mask):
    model.eval()
    with th.no_grad():
        embeddings = model.forward(g, features)
        indices , labels = resultSet_eval(embeddings,mask)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

## Define NN layers

In [12]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g,feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['vector'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = th.cat([feature,g.ndata['vector']],dim=1)
            g.ndata['vector'] = self.linear(h)
            return g.ndata['vector']
#             g.ndata['vector'] = feature
#             g.update_all(gcn_msg, gcn_reduce)
#             h = g.ndata['vector']
#             return self.linear(h)
        
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.layer1 = GCNLayer(300, 150)
#         self.layer2 = GCNLayer(150, 100)
    
#     def forward(self, g,features):
#          #x = F.relu(self.layer1(g, features))
#         x = F.leaky_relu(self.layer1(g, features))
#         #x = F.relu(self.layer2(g, x))
#         x = self.layer2(g, x)
#         return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(600, 300)
        self.layer2 = GCNLayer(600, 300)
        self.layer3 = nn.Linear(300, 300)
    
    def forward(self, g,features):
        x = F.leaky_relu(self.layer1(g,features))
        x = self.layer2(g, x)
        x = th.tanh(self.layer3(x))
        return x
    
    def forward_softmax(self, features):
        x = th.tanh(self.layer3(features))
        x = F.log_softmax(self.layer4(x),dim=1)
        return x
    

## Train

In [13]:
import time
import numpy as np
net = Net()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
loss_func = nn.CosineEmbeddingLoss(reduction='none')
for epoch in range(5):
    t0 = time.time()
    
    net.train()
    embeddings = net.forward(g, g.ndata['vector'])
    v1,v2,labels = resultSet_train(embeddings,train_mask)
    loss = loss_func(v1,v2, labels)
    
    optimizer.zero_grad()
    
    #loss.backward(retain_graph=True)
    loss.backward()
    optimizer.step()

    
    ##############################
#     v_softmax,labels_tipo = resultSet_train_softmax(embeddings,train_mask,g.ndata['tipo'])
#     embeddings_softmax = net.forward_softmax(v_softmax)
#     loss2 = F.nll_loss(embeddings_softmax, labels_tipo)
#     loss2.backward()
#     optimizer.step()
    
    
#     total_loss = loss2 + loss
#     total_loss.backward()
#     optimizer.step()
    ##############################
    
    
    
    dur.append(time.time() - t0)
    
    #acc = evaluate(net, g, embeddings.detach(), test_mask)
    acc = evaluate(net, g, g.ndata['vector'], test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Epoch 00000 | Loss 0.7486 | Test Acc 0.2711 | Time(s) 899.0080
Epoch 00001 | Loss 0.6646 | Test Acc 0.3756 | Time(s) 809.6798
Epoch 00002 | Loss 0.6173 | Test Acc 0.4701 | Time(s) 730.3303
Epoch 00003 | Loss 0.5473 | Test Acc 0.5821 | Time(s) 691.8118
Epoch 00004 | Loss 0.4454 | Test Acc 0.5970 | Time(s) 673.7709


In [29]:
result,label = resultSet_eval(g.ndata['vector'],test_mask)

In [36]:
#result_sum = th.sum(1-(result - label))
#label
ds1_set = list(set(df_matching["'dataset1_id'"]))
ds2_set = list(set(df_matching["'dataset2_id'"]))
print(len(ds1_set))
print(len(ds2_set))

201
201


In [4]:
import torch as th
hola = th.tensor([True, True, False])
th.sum(hola)

tensor(2)

In [18]:
1-cosine(embeddings[1].detach(), embeddings[233].detach())

0.9733333587646484

In [14]:
hola = list(filter(lambda x: x[2] == 1, test_mask))
print (len(hola)/len(test_mask))

0.24378109452736318


In [46]:
test_mask

[[374, 537, 0],
 [374, 413, 0],
 [374, 435, 0],
 [374, 494, 0],
 [374, 519, 0],
 [571, 524, 0],
 [576, 493, 0],
 [576, 404, 0],
 [576, 434, 0],
 [576, 512, 1],
 [417, 481, 0],
 [417, 538, 0],
 [417, 382, 1],
 [417, 426, 0],
 [417, 479, 1],
 [417, 489, 0],
 [418, 464, 1],
 [418, 496, 1],
 [418, 454, 0],
 [464, 385, 0],
 [466, 480, 0],
 [466, 487, 0],
 [466, 564, 0],
 [466, 438, 0],
 [466, 449, 0],
 [466, 529, 0],
 [470, 386, 0],
 [470, 465, 0],
 [476, 509, 1],
 [486, 496, 1],
 [486, 443, 0],
 [486, 490, 0],
 [491, 469, 1],
 [491, 548, 1],
 [491, 553, 0],
 [491, 415, 1],
 [491, 419, 0],
 [491, 449, 0],
 [495, 480, 1],
 [495, 525, 0],
 [495, 574, 1],
 [495, 402, 0],
 [495, 435, 1],
 [495, 462, 0],
 [496, 523, 0],
 [496, 535, 0],
 [496, 556, 1],
 [496, 468, 0],
 [496, 475, 0],
 [496, 489, 0],
 [501, 441, 0],
 [508, 488, 0],
 [508, 547, 0],
 [508, 382, 1],
 [508, 384, 1],
 [508, 414, 0],
 [508, 457, 0],
 [508, 467, 0],
 [508, 503, 0],
 [508, 565, 0],
 [515, 521, 0],
 [515, 506, 0],
 [515, 4

In [48]:
# v12,labels2 = resultSet_eval(embeddings.detach(),test_mask)
# v12 - labels2
1 - cosine(embeddings[374].detach(),embeddings[537].detach())

0.6918845772743225

In [ ]:
# import time
# import numpy as np
# net = Net()
# #g, features, labels, train_mask, test_mask

# optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
# dur = []
# for epoch in range(20):
#     if epoch >=3:
#         t0 = time.time()

#     net.train()
#     embeddings = net(g, g.ndata['vector'])
#     indices,labels = resultSet_train(embeddings.detach(),train_mask)
#     logp = F.logsigmoid(indices)
#     loss = F.nll_loss(logp, labels)
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     if epoch >=3:
#         dur.append(time.time() - t0)
    
#     acc = evaluate(net, g, embeddings.detach(), test_mask)
#     print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
#             epoch, loss.item(), acc, np.mean(dur)))

## Tests

In [ ]:
#get data from a specific node
g.nodes[0].data
#get data from nodes
g.ndata
#another way of accessing data from a node
g.ndata['tipo'][0]

In [ ]:
g.edges()

In [ ]:
g.has_edge_between(374,17619)

In [ ]:
g.ndata

In [ ]:
for n in g.nodes:
    print (n.data['tipo'])